<font size=6,font face = 'Times New Roman'>**Group Project**  
<font size=4,font face = 'Times New Roman'>学号+姓名

<font size=5,font face = 'Times New Roman'>**Import library**  

In [14]:
import gradio as gr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
import utils
from argparse import Namespace

args = Namespace()
args.generate_model="D:\\DDA4210\\facebookopt-1.3b"
args.util_model="D:\\DDA4210\\gpt"
args.use_gpu=True
args.prompt_max_length = None
args.max_new_tokens=200
args.gamma=0.25
args.delta=2.0
args.detection_z_threshold=4.0
args.generation_seed=42
args.use_sampling=True
args.sampling_temp=0.7
args.n_beams=1
args.normalizers=""
args.ignore_repeated_ngrams=False

model, tokenizer, device, pplmodel, ppltokenizer = utils.load_model(args)

input_text = utils.load_prompts()
# input_text = utils.load_prompts()
print('prompt: ' + input_text)

gpu success
prompt: how does the eye-sight-measuring machine at the glasses shop actually give you the exact number of your eyesight?


<font size=6>第一个tap 用来简单prompt

In [16]:
def random_choose():
    return utils.load_prompts()

<font size=6>第二个tap 第一部分 生成文本并且hightlight green list

In [17]:
# 输入是生成的文本和得到的green list 得到的结果是对应的高亮操作
# 需要把output改成return （text，green list）
def Hight(text, word_list):
    output = []
    words = text.split(' ')
    print(words)
    for word in words:
        print(word)
        if word.lower() in word_list:
            output.append((word, '+'))
        else:
            output.append((word, None))
    return output

In [18]:
def WM(input_text):
    global args
    global with_wm
    without_wm, with_wm= utils.generate(input_text, 
                                    args, 
                                    model=model, 
                                    device=device, 
                                    tokenizer=tokenizer)
    global wl
    wl = utils.detect(with_wm, 
                          args, 
                          device=device, 
                          model = model,
                          tokenizer=tokenizer)[4][1]
    return Hight(without_wm,wl),Hight(with_wm,wl)

In [19]:
def attack():
    at_1 = utils.paraphrasing_attack(with_wm)
    at_2 = utils.substitution_attack(with_wm)
    return Hight(at_1, wl),Hight(at_2, wl)

In [20]:
# def WM(a,b):
#     word_list = ['The', 'was', 'devised', 'Phil', 'as', 'attack', 'weak', \
#                  'side', 'The', 'consists', 'power', 'guard', 'The', 'power', \
#                 'primary', 'scorer', 'while', 'primary', 'rebound', 'The', \
#                 'guard', 'primary', 'play', 'while', 'power', 'primary', \
#                 'distributor', 'The', 'was', 'devised', 'Phil', 'as', \
#                 'attack', 'weak', 'side', 'The', 'consists', 'power', \
#                 'guard', 'The', 'power', 'primary', 'scorer', 'while',\
#                  'primary', 'rebound', 'The', 'guard', 'primary', 'play', \
#                 'while', 'power', 'primary', 'distributor', 'The', 'was', \
#                 'devised', 'Phil', 'as', 'attack', 'weak', 'side', 'The', \
#                 'consists', 'power', 'guard', 'The', 'power', 'primary', \
#                 'scorer', 'while', 'primary', 'rebound', 'The', 'guard', \
#                 'primary', 'play', 'while']
#     str1 ="""The triangle offense was devised by Phil Jackson as a way to attack the weak side of the defense. 
#     The triangle consists of a power forward, a center, and a guard. 
#     The power forward is the primary scorer while the center is the main rebounder. 
#     The guard is the playmaker while the rebounder is the distributor.
#     In order to execute the triangle offense, you need to be able to score from the weakside of the court. 
#     The primary scorer should be able score from both the weak and strong side of his position. 
#     The playmaker should be a playmaker that can create his own shot while the rebounding guard should be the primary rebounder while the power forward should be his primary scorer.
#     The primary scorer needs to be a scorer that can score from either side of him while the playmakers needs to create his shot while rebounding. 
#     The rebounding power forward needs to rebound while the primary scorers needs to score while rebinding."""
#     str2 ="""must have good ball movement and communication between players to effectively run the triangle offense. 
#     The power forward, center, and guard must work together to create scoring opportunities and exploit the weaknesses of the defense. 
#     It is essential for players to understand their roles and responsibilities within the offense in order to be successful."""
#     if b == 'simple':
#         return Hight(str1,word_list)
#     if b == '"Paraphrasing Attack"':
#         return Hight(str2,word_list)

<font size=6>第二个tap 第二部分 生成evaluation

In [21]:
# 正常版
def Evaluation(text,operation):
    global args
    analysis = {}
    analysis['gamma'] = args.gamma
    analysis['delta'] = args.delta
    analysis['z_threshold'] = args.detection_z_threshold
    without_wm, with_wm= utils.generate(text, 
                                        args, 
                                        model=model, 
                                        device=device, 
                                        tokenizer=tokenizer)
    if operation == 'simple':
        with_wm_detection = utils.detect(with_wm, 
                                args, 
                                device=device, 
                                model = model,
                                tokenizer=tokenizer)
        analysis['T'] = with_wm_detection[0][1]
        analysis['z'] = with_wm_detection[2][1]
        analysis['p'] = with_wm_detection[3][1]
        analysis['prediction'] = with_wm_detection[6][1]
        ppl_with_wm = utils.compute_ppl(with_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
        analysis['ppl'] = ppl_with_wm
    

    if operation == "Paraphrasing Attack":
        rewritten_wm = utils.paraphrasing_attack(with_wm)
        rewritten_with_wm_detection = utils.detect(rewritten_wm, 
                                        args, 
                                        device=device, 
                                        model = model,
                                        tokenizer=tokenizer)
        analysis['T'] = rewritten_with_wm_detection[0][1]
        analysis['z'] = rewritten_with_wm_detection[2][1]
        analysis['p'] = rewritten_with_wm_detection[3][1]
        analysis['prediction'] = rewritten_with_wm_detection[6][1]
        ppl_rewritten_with_wm = utils.compute_ppl(rewritten_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
        analysis['ppl'] = ppl_rewritten_with_wm

    if operation == "Paraphrasing Attack":
        rewritten_wm = utils.substitution_attack(with_wm)
        rewritten_with_wm_detection = utils.detect(rewritten_wm, 
                                        args, 
                                        device=device, 
                                        model = model,
                                        tokenizer=tokenizer)
        analysis['T'] = rewritten_with_wm_detection[0][1]
        analysis['z'] = rewritten_with_wm_detection[2][1]
        analysis['p'] = rewritten_with_wm_detection[3][1]
        analysis['prediction'] = rewritten_with_wm_detection[6][1]
        ppl_rewritten_with_wm = utils.compute_ppl(rewritten_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
        analysis['ppl'] = ppl_rewritten_with_wm


    df = pd.DataFrame(analysis)
    return df

In [22]:
# 开天眼版
def table(analysis,operation):
    data = {'gamma': [0.25], 
            'delta': [2.0], 
            'z_threshold': [4.0], 
            'T': ['84'], 
            'z': ['5.55'], 
            'p': ['1.41e-08'], 
            'prediction': ['True']
            }
    df = pd.DataFrame(data)
    return df

<font size=6>第三个tap 生成图像

In [23]:
# 正常版
def plot(A,B,C):
    a = A.split(' ')# a 是gamma的list
    b = B.split(' ')# b 是 delta的list
    c = C.split(' ')# c是 z-xx的list
    

In [24]:
# 不是开天眼的测试版
def generate_plot(a,b,c):
    x = np.linspace(0, 10, 100)
    y = np.sin(x)
    plt.plot(x, y)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Sine Wave')
    plt.grid(True)
    return plt

<font size=6>总函数

In [25]:
with gr.Blocks() as demo:
    # with gr.Tab("Raw material"): # 新建一个 Tab
        # prompt= gr.Textbox(lines=3,
        #                    value=input_text, 
        #                    label="Prompt")
        # b = gr.Button("Generate")
        # o = gr.Textbox(lines=3,
        #                 # value=original_answer, 
        #                 label="Original answer")
    # gr.Markdown("Prompt:"+ input_text) 
    # gr.Markdown("Original_answer:"+ original_answer)
    with gr.Tab("Watermark Generation"): # 新建一个 Tab
        text_input = gr.Textbox(lines=5, placeholder="Text questions Here...", label="My Prompting")
        Random_question_button = gr.Button("Random question")
        # Random_question = gr.Textbox(lines=3, label="Original answer")
        # input_prompt = gr.Textbox(lines=3, label="input_prompt")
        # radio = gr.Radio(["simple", "Attack"], label='Model Choice')

        Generate_text_button = gr.Button("Generate")
        with gr.Row():
            wmtext_output_1 = gr.HighlightedText(
                            label="Generate without WM",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
            wmtext_output_2 = gr.HighlightedText(
                            label="Generate with WM",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
        # wmtextboxes_row = gr.Column([wmtext_output_1, wmtext_output_2])
        Attack_text_button = gr.Button("Attack")
        with gr.Row():
            text_attack_1 = gr.HighlightedText(
                            label="Parapharising attack",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
            text_attack_2 = gr.HighlightedText(
                            label="Substitution attack",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
        # attackboxes_row = gr.Column([text_attack_1, text_attack_2])
        # Evaluate_button = gr.Button("Evaluate")
        # Evaluation_output = gr.DataFrame(label= 'Evaluation Table',interactive=True,wrap=True)
        
    # with gr.Tab("Sensitivity Analysis"): # 新建一个 Tab
    #     parameter = [gr.Textbox(lines=1,placeholder="Text gamma Here...", label="gamma"),
    #                  gr.Textbox(lines=1,placeholder="Text delta Here...", label="delta"),
    #                  gr.Textbox(lines=1,placeholder="Text z threshold Here...", label="z threshold")]
    #     image_button = gr.Button('visualization')
    #     image_output= gr.Plot()
    # with gr.Accordion("Open for More!"): # 可折叠的组件
    #     gr.Markdown("Look at me...")

    # b.click(initial, inputs=prompt, outputs=o) # 按钮绑定相应的槽函数
    Random_question_button.click(random_choose, outputs=text_input)
    Generate_text_button.click(WM, inputs=text_input, outputs=[wmtext_output_1, wmtext_output_2]) # 按钮绑定相应的槽函数
    Attack_text_button.click(attack, outputs=[text_attack_1, text_attack_2]) # 按钮绑定相应的槽函数
    # Evaluate_button.click(table, inputs=text_input, outputs=Evaluation_output) # 按钮绑定相应的槽函数
    # image_button.click(plot, inputs=parameter, outputs=image_output) # 按钮绑定相应的槽函数

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


['', 'I', 'know', "it's", 'fiction', 'but', 'it', 'still', 'gets', 'to', "me\nWe're", 'not', 'talking', 'western', 'heroes', 'so', 'I', "don't", 'know', 'what', "you're", 'talking', 'about.', '', '', '', 'And', 'action', 'movies', 'are', 'like', '90%', 'human', 'stories,', 'for', 'most', 'people', 'they', "don't", 'care.', '', "That's", 'why', 'when', 'I', 'saw', '"Guardians', 'of', 'the', 'Galaxy', 'and', 'Thor"', 'I', 're-watched', 'the', 'film', 'soundtrack', 'while', 'relaxing,', 'because', 'at', 'that', 'point', 'it', 'made', 'more', 'overall', 'sense', 'to', 'me', 'than', 'the', 'actual', 'movie', 'they', 'were', 'watching.\nI', 'somehow', 'missed', 'thatggggg', 'one', 'night,', 'guess', 'I', 'should', 'give', 'it', 'another', 'go.', 'And', 'hey', 'I', 'like', 'superheroes', 'and', 'would', 'like', 'to', 'own', 'a', 'home', 'deagle', 'but', 'maybe', 'save', 'up', 'for', 'a', 'duracell?', 'idk\nA', 'blow', 'job', 'and', 'some', 'kidneys', 'from', 'Natalie', 'Portman', 'should', 'd

Traceback (most recent call last):
  File "d:\miniconda\lib\site-packages\gradio\queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "d:\miniconda\lib\site-packages\gradio\route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "d:\miniconda\lib\site-packages\gradio\blocks.py", line 1741, in process_api
    result = await self.call_function(
  File "d:\miniconda\lib\site-packages\gradio\blocks.py", line 1296, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "d:\miniconda\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "d:\miniconda\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "d:\miniconda\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, 